In [9]:
%load_ext autoreload
%autoreload 2

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Subset
from torchvision import datasets, transforms
from complexPyTorch.complexLayers import ComplexBatchNorm2d, ComplexConv2d, ComplexLinear, ComplexConv2dGauss
from complexPyTorch.complexLayers import ComplexDropout2d, NaiveComplexBatchNorm2d
from complexPyTorch.complexLayers import ComplexBatchNorm1d
from complexPyTorch.complexFunctions import complex_relu, complex_max_pool2d

In [13]:
batch_size = 64
n_train = 1000
n_test = 100
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
train_set = datasets.MNIST('../data', train=True, transform=trans, download=True)
train_set = Subset(train_set, torch.arange(n_train))
test_set = datasets.MNIST('../data', train=False, transform=trans, download=True)
test_set = Subset(test_set, torch.arange(n_test))

train_loader = torch.utils.data.DataLoader(train_set, batch_size= batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size= batch_size, shuffle=True)

In [32]:
class ComplexNet(nn.Module):
    
    def __init__(self):
        super(ComplexNet, self).__init__()
        self.conv1 = ComplexConv2dGauss(1, 10, 5, 1)
        self.bn2d  = ComplexBatchNorm2d(10, track_running_stats = False)
        self.conv2 = ComplexConv2dGauss(10, 20, 5, 1)
        self.fc1 = ComplexLinear(4*4*20, 500)
        self.dropout = ComplexDropout2d(p = 0.3)
        self.bn1d  = ComplexBatchNorm1d(500, track_running_stats = False)
        self.fc2 = ComplexLinear(500, 10)
             
    def forward(self,x):
        x = self.conv1(x.type(torch.cfloat))
        x = complex_relu(x)
        x = complex_max_pool2d(x, 2, 2)
        x = self.bn2d(x)
        x = self.conv2(x)
        x = complex_relu(x)
        x = complex_max_pool2d(x, 2, 2)
        x = x.view(-1,4*4*20)
        x = self.fc1(x)
        x = self.dropout(x)
        x = complex_relu(x)
        x = self.bn1d(x)
        x = self.fc2(x)
        x = x.abs()
        x =  F.log_softmax(x, dim=1)
        return x
    
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ComplexNet().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=5e-3, momentum=0.9)

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target =data.to(device).type(torch.complex64), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train\t Epoch: {:3} [{:6}/{:6} ({:3.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(data), 
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader), 
                loss.item())
            )
            
def test(model, device, test_loader, optimizer, epoch):
    model.eval()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device).type(torch.complex64), target.to(device)
        output = model(data)
        loss = F.nll_loss(output, target)
        if batch_idx % 100 == 0:
            print('Test\t Epoch: {:3} [{:6}/{:6} ({:3.0f}%)]\tLoss: {:.6f}'.format(
                epoch,
                batch_idx * len(data), 
                len(test_loader.dataset),
                100. * batch_idx / len(test_loader), 
                loss.item())
            )

In [5]:
# Run training on 4 epochs
for epoch in range(4):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader, optimizer, epoch)

/Users/skandel/.conda/envs/torchdiff/lib/python3.10/site-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Train	 Epoch:   0 [     0/  1000 (  0%)]	Loss: 2.594220
Test	 Epoch:   0 [     0/   100 (  0%)]	Loss: 0.482387
Train	 Epoch:   1 [     0/  1000 (  0%)]	Loss: 0.916131
Test	 Epoch:   1 [     0/   100 (  0%)]	Loss: 0.402028
Train	 Epoch:   2 [     0/  1000 (  0%)]	Loss: 0.608397
Test	 Epoch:   2 [     0/   100 (  0%)]	Loss: 0.198328
Train	 Epoch:   3 [     0/  1000 (  0%)]	Loss: 0.130472
Test	 Epoch:   3 [     0/   100 (  0%)]	Loss: 0.022836


In [33]:
# Run training on 4 epochs
for epoch in range(4):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader, optimizer, epoch)

RuntimeError: imag is not implemented for tensors with non-complex dtypes.

In [31]:
torch.cfloat

torch.complex64

In [6]:
l1 = nn.Conv1d

In [7]:
l1.forward??

Signature: l1.forward(self, input: torch.Tensor) -> torch.Tensor
Docstring:
Defines the computation performed at every call.

Should be overridden by all subclasses.

.. note::
    Although the recipe for forward pass needs to be defined within
    this function, one should call the :class:`Module` instance afterwards
    instead of this since the former takes care of running the
    registered hooks while the latter silently ignores them.
Source:   
    def forward(self, input: Tensor) -> Tensor:
        return self._conv_forward(input, self.weight, self.bias)
File:      ~/.conda/envs/torchdiff/lib/python3.10/site-packages/torch/nn/modules/conv.py
Type:      function


In [8]:
l1.weight

AttributeError: type object 'Conv1d' has no attribute 'weight'